In [6]:
%%bash

cd src
ls .git/config
cat .git/config

.git/config
[core]
	bare = false
[remote "origin"]
	url = https://github.com/meddash-cloud/cicd-projects-examples.git
	fetch = +refs/heads/*:refs/remotes/origin/*
[branch "main"]
	remote = origin
	merge = refs/heads/main
[branch "datapipelines-019"]
	remote = origin
	merge = refs/heads/datapipelines-019


In [7]:
%%bash

git clone https://github.com/meddash-cloud/cicd-projects-examples.git cicd-projects
cd cicd-projects
ls

Cloning into 'cicd-projects'...


cicd-ex-001
cicd-ex-002


In [10]:
%%bash
cd cicd-projects
git checkout -b datapipelines-021
git branch

Switched to a new branch 'datapipelines-021'


* datapipelines-021
  main


In [12]:
%%bash
cd cicd-projects
mkdir datapipelines-021
cd datapipelines-021


In [13]:
!pip3 install minio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.9 MB/s eta 0:00:00


In [25]:
%%bash
## create dummy files (sleep 1 to generated random string)
echo `date| md5sum` > /tmp/file01 && sleep 1
echo `date| md5sum` > /tmp/file02 && sleep 1
echo `date| md5sum` > /tmp/file03 && sleep 1
echo `date| md5sum` > /tmp/file04 && sleep 1
echo `date| md5sum` > /tmp/file05
cat /tmp/file0*

e9e66f405890fe52bd0d51f8b710844f -
63ab17ca082c2cf2df594f8d2c4be954 -
f97673e6abcac710410fc4630d25d86d -
2a03e143e6be5e2f870d944988e2ec66 -
d46a88c4215fb3ededb0808ac282ab7d -


In [14]:
from minio import Minio

def main():
    # Create a client with the MinIO server playground, its access key
    # and secret key.
    client = Minio(
        "play.min.io",
        access_key="Q3AM3UQ867SPQQA43P2F",
        secret_key="zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG",
    )

    # Make 'asiatrip' bucket if not exist.
    found = client.bucket_exists("asiatrip")
    if not found:
        client.make_bucket("asiatrip")
    else:
        print("Bucket 'asiatrip' already exists")

    # Upload '/home/user/Photos/asiaphotos.zip' as object name
    # 'asiaphotos-2015.zip' to bucket 'asiatrip'.
    client.fput_object(
        "asiatrip", "asiaphotos-2015.zip", "/home/user/Photos/asiaphotos.zip",
    )
    print(
        "'/home/user/Photos/asiaphotos.zip' is successfully uploaded as "
        "object 'asiaphotos-2015.zip' to bucket 'asiatrip'."
    )

In [37]:
%%bash
env |grep -i minio

MINIO_SERVICE_SERVICE_PORT_HTTP=9000
MINIO_SERVICE_PORT_9000_TCP_ADDR=10.152.183.238
MINIO_SERVICE_PORT=tcp://10.152.183.238:9000
MINIO_SERVICE_PORT_9000_TCP_PROTO=tcp
MINIO_SERVICE_PORT_9000_TCP_PORT=9000
MINIO_SERVICE_PORT_9000_TCP=tcp://10.152.183.238:9000
MINIO_SERVICE_SERVICE_HOST=10.152.183.238
MINIO_SERVICE_SERVICE_PORT=9000


In [40]:
%%bash
echo minio address:
echo http://$MINIO_SERVICE_SERVICE_HOST:$MINIO_SERVICE_SERVICE_PORT_HTTP

minio address:
http://10.152.183.238:9000


In [45]:
import os
minio_host, minio_port = os.environ["MINIO_SERVICE_SERVICE_HOST"], os.environ["MINIO_SERVICE_SERVICE_PORT_HTTP"]
minio_url= "{}:{}".format(minio_host, minio_port)
print(minio_url)

10.152.183.238:9000


In [53]:
from minio import Minio
from minio.error import S3Error

BUCKET_NAME="datapipeline-021"

config = {
    "endpoint": minio_url,
    "access_key": "minio",
    "secret_key": "minio123",
    "secure": False,
    }


# Create a client with the MinIO server playground, its access key
# and secret key.
print ("connecting to minio {}".format(minio_url))
minio_client = Minio(**config)

print("try to find bucket {}".format(BUCKET_NAME))
found = minio_client.bucket_exists(BUCKET_NAME)
print("found", found)
if not found:
    minio_client.make_bucket(BUCKET_NAME)
else:
    print("Bucket '{}' already exists".format(BUCKET_NAME))

# # Upload '/home/user/Photos/asiaphotos.zip' as object name
# # 'asiaphotos-2015.zip' to bucket 'asiatrip'.
# client.fput_object(
# "asiatrip", "asiaphotos-2015.zip", "/home/user/Photos/asiaphotos.zip",
# )
# print(
# "'/home/user/Photos/asiaphotos.zip' is successfully uploaded as "
# "object 'asiaphotos-2015.zip' to bucket 'asiatrip'."
# )


connecting to minio 10.152.183.238:9000
try to find bucket datapipeline-021
found True
Bucket 'datapipeline-021' already exists


In [57]:
import glob
import os
files=glob.glob("/tmp/file0*")
for filepath in files:
    minio_client.fput_object(BUCKET_NAME, os.path.basename(filepath), filepath)

In [64]:

def download_path(filename):
    return "/tmp/minio_download_{}".format(filename)

for item in minio_client.list_objects(BUCKET_NAME,recursive=False):
    print(item.object_name)
    obj=minio_client.fget_object(BUCKET_NAME,item.object_name,download_path(item.object_name))
    print(obj)


file01
file02
file03
file04
file05


In [68]:
%%bash
for f in `ls /tmp/minio_download*`; do
    echo $f
    cat $f
done


/tmp/minio_download_file01
e9e66f405890fe52bd0d51f8b710844f -
/tmp/minio_download_file02
63ab17ca082c2cf2df594f8d2c4be954 -
/tmp/minio_download_file03
f97673e6abcac710410fc4630d25d86d -
/tmp/minio_download_file04
2a03e143e6be5e2f870d944988e2ec66 -
/tmp/minio_download_file05
d46a88c4215fb3ededb0808ac282ab7d -
